# Выбираем модель

In [1]:
from common import load_data, validation, compare_models
from models import get_all_models

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
X_train, Y_train, X_test, Y_test = load_data()

In [3]:
num_train, height, width, depth = X_train.shape
num_classes = Y_train.shape[1]

In [4]:
models = get_all_models(height, width, depth, num_classes)

In [0]:
res = compare_models(models, X_train, Y_train, X_test, Y_test, start_from=35)

  28   0.934634   0.716800   <class 'rdu_nn_cifar.models.VggLikeModel'> conv_params=[{'conv_count': 2, 'filters': 16, 'activation': 'tanh', 'dropout_rate': 0.4, 'kernel_regularizer': <keras.regularizers.L1L2 object at 0x7f884c776940>}, {'conv_count': 2, 'filters': 32, 'activation': 'tanh', 'dropout_rate': 0.4, 'kernel_regularizer': <keras.regularizers.L1L2 object at 0x7f884c7768d0>}, {'conv_count': 2, 'filters': 64, 'activation': 'tanh', 'dropout_rate': 0.4, 'kernel_regularizer': <keras.regularizers.L1L2 object at 0x7f884c776dd8>}] dense_dropout_rate=0.5 dense_size=[512]
  29   0.912153   0.720900   <class 'rdu_nn_cifar.models.VggLikeModel'> conv_params=[{'conv_count': 3, 'filters': 16, 'activation': 'tanh', 'dropout_rate': 0.4, 'kernel_regularizer': <keras.regularizers.L1L2 object at 0x7f884c4f3c50>}, {'conv_count': 3, 'filters': 32, 'activation': 'tanh', 'dropout_rate': 0.4, 'kernel_regularizer': <keras.regularizers.L1L2 object at 0x7f884c507940>}, {'conv_count': 3, 'filters': 64, 'a

In [0]:
res

# Дотрейниваем сеть


In [5]:
model_ix = 35

In [ ]:
model = models[model_ix]

In [ ]:
validation(model, X_train, Y_train, X_test, Y_test, epochs=20, verbose=1)

Model: 
4.4920469131469725 0.1
Train on 45000 samples, validate on 5000 samples
Epoch 1/20
 4384/45000 [=>............................] - ETA: 1:58:37 - loss: 5.4593 - acc: 0.1357

# Делаем сабмит

In [0]:
import numpy as np
import pandas as pd

In [19]:
x_valid = np.load('/content/test.npy')
x_valid.shape

(2000, 32, 32, 3)

In [0]:
x_valid = x_valid.astype('float32')
max_x_values = 256
x_valid /= max_x_values  # Normalise data to [0, 1] range

In [0]:
predict = model.predict(x_valid)

In [22]:
pd_predict = pd.DataFrame(predict)
pd_predict.head()

,0,1,2,3,4,5,6,7,8,9
0,1.789034e-01,4.161344e-03,8.637774e-02,1.189657e-01,3.754953e-02,1.091564e-01,5.945502e-03,4.013829e-01,3.016956e-03,5.454049e-02
1,4.630393e-09,5.330252e-09,2.874460e-03,2.464778e-03,1.654522e-07,9.945354e-01,1.255259e-05,1.124890e-04,1.180288e-08,9.870973e-08
2,7.165512e-05,4.307057e-07,3.283823e-02,3.965079e-03,2.300791e-02,6.464711e-01,7.117366e-04,2.929336e-01,8.841874e-09,1.844541e-07
3,1.230051e-02,6.324655e-03,8.708184e-02,4.050241e-01,4.735753e-02,2.454573e-01,3.779888e-02,1.355391e-01,1.293396e-02,1.018220e-02
4,2.126846e-03,7.353626e-04,5.684761e-07,4.985782e-07,5.503691e-08,2.003356e-08,2.554098e-09,1.494241e-08,9.923425e-01,4.794078e-03


In [0]:
pd_predict = pd_predict.idxmax(axis=1)

In [0]:
pd_predict = pd_predict.reset_index()

In [0]:
pd_predict.columns = ['Id', 'Category']

In [26]:
file_name = "model_{}.csv".format(model_ix)
print(file_name)
pd_predict.to_csv(file_name, header=1, index=0)

model_30.csv


In [27]:
!head $file_name

Id,Category
0,7
1,5
2,5
3,3
4,8
5,5
6,7
7,6
8,9


In [0]:
import re

In [0]:
s_message = "" + re.sub(r'[^\w\d\s]','_', model.name)

In [30]:
!kaggle competitions submit -f $file_name -m "$s_message" -c 'another-cifar-10'

Successfully submitted to CIFAR-10